In [1]:
import nibabel as nib
import numpy as np
import os
import glob
import SimpleITK as sitk

In [28]:
base_dir = r"G:\Semester 5\Data Science and Engineering Project\Codes\MRI-Enhance-3D\DataSet"

# Use glob to find all files in the directory
file_paths = glob.glob(os.path.join(base_dir, '*'))

In [26]:
def pipeline_for_one(file_path):
    # Extract the file name from the file path
    file_name = os.path.basename(file_path)

    # Load the image
    nib_file = nib.load(file_path)

    # Extract the header from the nifti file
    header = nib_file.header

    # Print Header Information
    # print(header)


    ### Update the affine matrix in the header ###
    
    # Extract the voxel spacing from the header
    voxel_spacing = header.get_zooms()

    # Extract the qoffsets from the header
    qoffset_x = header['qoffset_x']
    qoffset_y = header['qoffset_y']
    qoffset_z = header['qoffset_z']

    # Create a new affine matrix
    new_affine = np.array([
    [voxel_spacing[0], 0, 0, qoffset_x],
    [0, voxel_spacing[1], 0, qoffset_y],
    [0, 0, voxel_spacing[2], qoffset_z],
    [0, 0, 0, 1]
    ])

    # Override the affine matrix in the header
    new_header = header.copy()

    new_header['srow_x'] = new_affine[0,:]
    new_header['srow_y'] = new_affine[1,:]
    new_header['srow_z'] = new_affine[2,:]

    # Save the new nifti file
    updated_img = nib.Nifti1Image(nib_file.get_fdata(), new_affine, new_header)

    # Print the new header
    # print(updated_img.header)


    ### Downsample the image ###
    # Downsample the image by a factor of 5
    downsampled_image = updated_img.slicer[::5,:,:]

    # Print the header of the downsampled image
    print(downsampled_image.header)


    ### Resample the images ###
    
    # Define the interpolator
    interpolator = sitk.sitkLinear

    # Define the new voxel spacing
    new_spacing = [1, 1, 1]

    # Convert the NIfTI image data to a SimpleITK image
    img_data = updated_img.get_fdata()
    volume = sitk.GetImageFromArray(img_data.astype(np.float32))

    original_spacing = volume.GetSpacing()
    original_size = volume.GetSize()

    new_size = [int(round(osz*ospc/nspc)) for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)]
    resample_volume = sitk.Resample(volume, new_size, sitk.Transform(), interpolator, volume.GetOrigin(), new_spacing, volume.GetDirection(), 0.0, volume.GetPixelID())

    sitk.WriteImage(resample_volume, f"{base_dir}\\{file_name}_resampled.nii.gz")




In [27]:
pipeline_for_one(file_paths[0])

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3  35 256 256   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.       4.999991 1.       1.       1.       1.       1.       1.      ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'TE=3.2;Time=175927.760'
aux_file        : b'imgComments'
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 0.0
qoffset_x       : -96.71713
qoffset_y    

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 256 256 171   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [1. 1. 1. 1. 0. 0. 0. 0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code      : scanner
quatern_b       : 0.0
quatern_c       : 0.0
quatern_d       : 1.0
qoffset_x       : -0.0
qoffset_y       : -0.0
qoffset_z       : 0.0
srow_x          : [-1.  0.  0. -0.]
srow_y         